# Lesson 2: Talking to the PDS API

Welcome back! In our last notebook, we explored how an LLM can use RAG (Retrieval-Augmented Generation) to figure out exactly *what* we want to search for in the NASA PDS. We successfully mapped fuzzy human concepts (like "defrosted dunes") to rigid database filters (like `dark dune`).

But our LLM is currently just talking to us. It isn't actually getting any images!

In this notebook, we will focus on the LLM's second job, which is actually **retrieving** the images. We are going to teach the LLM how to use an API to fetch the images we want. 

### Wait, what is an API?
If you've never used an **API** (Application Programming Interface) before, think of it like a waiter at a restaurant. 
1. You (the client) sit at the table and look at the menu. 
2. You can't just walk into the kitchen and start cooking. Instead, you give your order to the waiter (the API).
3. The waiter takes your specific order to the kitchen (the server/database), the kitchen prepares it, and the waiter brings your food back to you.

Instead of food, our API deals with data. We will send a request to the NASA PDS server, and it will hand us back the image data we asked for.

### Speaking the Server's Language: Lucene
When you type a search into the Atlas IV website, the site uses a query language called **Lucene** to filter millions of images. Lucene is just a specific, structured way to format a search. 

For example, if we want images from the Mars Reconnaissance Orbiter (MRO) where the sun is high in the sky (Solar Latitude between 0 and 80 degrees), the Lucene query looks like this:

`gather.common.mission:"mro" AND pds3_label.SOLAR_LATITUDE:[0 TO 80]`

Let's write a quick Python script to send this exact Lucene query to the PDS Search API!

In [18]:
import requests
import json

# 1. The PDS Atlas Search Endpoint
# This is the "waiter" we are handing our order to.
search_url = "https://pds-imaging.jpl.nasa.gov/api/search/atlas/_search"

# 2. Our Lucene Query
# In a full AI application, the LLM would generate this string for us based on the user's prompt!
lucene_query = 'gather.common.mission:"mro" AND pds3_label.SOLAR_LATITUDE:[0 TO 80]'

# 3. The Payload
# The server requires us to format our request in a specific JSON structure.
# We also limit the 'size' to 3 so we don't overwhelm our notebook with hundreds of results.
payload = {
    "query": {
        "query_string": {
            "query": lucene_query
        }
    },
    "size": 3 
}

# We have to tell the server we are sending JSON data
headers = {"Content-Type": "application/json"}

print(f"Sending our Lucene query to the PDS Atlas...\n")

# 4. Make the POST request
response = requests.post(search_url, json=payload, headers=headers)

# 5. Check if it worked and print the results!
if response.status_code == 200:
    data = response.json()
    
    # The PDS API returns a deeply nested JSON response. 
    # The actual records live inside: hits -> hits -> _source
    hits = data.get("hits", {}).get("hits", [])
    
    print(f"Success! Found {len(hits)} matching records. Here are their unique URIs:")
    
    # Let's save the first URI to use in the next step
    first_uri = None 
    
    for hit in hits:
        uri = hit.get("_source", {}).get("uri", "No URI found")
        print(f"- {uri}")
        if first_uri is None:
            first_uri = uri
else:
    print(f"Uh oh! The request failed with status code: {response.status_code}")
    print(response.text)

Sending our Lucene query to the PDS Atlas...

Success! Found 3 matching records. Here are their unique URIs:
- atlas:pds3:mro:mars_reconnaissance_orbiter:/HiRISE/EDR/ESP/ORB_058400_058499/ESP_058410_2210/ESP_058410_2210_BG12_0.IMG
- atlas:pds3:mro:mars_reconnaissance_orbiter:/HiRISE/EDR/ESP/ORB_058200_058299/ESP_058227_2050/ESP_058227_2050_RED8_0.IMG
- atlas:pds3:mro:mars_reconnaissance_orbiter:/HiRISE/EDR/ESP/ORB_058400_058499/ESP_058412_2325/ESP_058412_2325_RED5_0.IMG


Great, we found the unique URI for our image. However, the URIs alone don't need us to the image. We *could* write a bunch of complex Python code to download the raw scientific `.IMG` file and render the math array into a picture... 

But NASA has already done the hard work for us! 

The PDS Atlas has a built-in web viewer. All we have to do is take our URI and attach it to the end of their viewer URL. This is exactly what a smart AI Agent would do: find the record, and give the user a simple, clickable link to view it in their browser.

In [19]:
# The base URL for the Atlas Record Viewer
viewer_base_url = "https://pds-imaging.jpl.nasa.gov/tools/atlas/record"

# We simply attach our URI as a query parameter (?uri=...)
clickable_link = f"{viewer_base_url}?uri={first_uri}"

print("Success! Here is the direct link to view your Mars image:")
print(clickable_link)

Success! Here is the direct link to view your Mars image:
https://pds-imaging.jpl.nasa.gov/tools/atlas/record?uri=atlas:pds3:mro:mars_reconnaissance_orbiter:/HiRISE/EDR/ESP/ORB_058400_058499/ESP_058410_2210/ESP_058410_2210_BG12_0.IMG


Success! We have successfully demonstrated the use of the PDS API! In the next lesson, you will work on making the LLM use it to return URLs. 